In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')


Mounted at /content/drive/


In [3]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/archive.zip", 'r')
zip_ref.extractall("/content/tmp")
zip_ref.close()

In [4]:
human_training="/content/tmp/human-and-non-human/training_set/training_set/humans"
non_human_training="/content/tmp/human-and-non-human/training_set/training_set/non-humans"
human_test="/content/tmp/human-and-non-human/test_set/test_set/humans"
non_human_test="/content/tmp/human-and-non-human/test_set/test_set/non-humans"



In [5]:
import os #total images length
print("training_human_data:",len(os.listdir(human_training)))
print("training_NON_human_data:",len(os.listdir(non_human_training)))

print("test_human_data:",len(os.listdir(human_test)))


print("test_NON_human_data:",len(os.listdir(non_human_test)))





training_human_data: 4012
training_NON_human_data: 4006
test_human_data: 1415
test_NON_human_data: 1309


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
training_dir="/content/tmp/human-and-non-human/training_set/training_set/"
test_dir="/content/tmp/human-and-non-human/test_set/test_set/"
# Use ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen=ImageDataGenerator(rescale=1./255)

train_gen=train_datagen.flow_from_directory(training_dir,class_mode='binary',batch_size=32,target_size=(224,224))
test_gen=test_datagen.flow_from_directory(test_dir,class_mode='binary',batch_size=32,target_size=(224,224))





Found 8017 images belonging to 2 classes.
Found 2723 images belonging to 2 classes.


In [9]:
resnet50=tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',

    input_shape=(224,224,3),

    classes=1,

)
for layer in resnet50.layers:
  layer.trainable=False

94765736/94765736 [==============================] - 0s 0us/step


In [10]:
model=tf.keras.Sequential([
    resnet50,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(1,activation='sigmoid')




])

In [11]:
model.compile(optimizer=tf.keras.optimizers.SGD(),loss='binary_crossentropy',metrics=["accuracy"])

In [12]:
batch_size=32

In [13]:
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=10,
    validation_data=test_gen,
    validation_steps=test_gen.samples // batch_size

)


Epoch 1/10
250/250 [==============================] - 127s 470ms/step - loss: 0.6920 - accuracy: 0.5568 - val_loss: 0.6942 - val_accuracy: 0.4607
Epoch 2/10
250/250 [==============================] - 118s 471ms/step - loss: 0.6517 - accuracy: 0.6084 - val_loss: 0.6764 - val_accuracy: 0.5103
Epoch 3/10
250/250 [==============================] - 117s 467ms/step - loss: 0.6335 - accuracy: 0.6217 - val_loss: 0.6743 - val_accuracy: 0.5529
Epoch 4/10
250/250 [==============================] - 116s 464ms/step - loss: 0.6230 - accuracy: 0.6358 - val_loss: 0.6803 - val_accuracy: 0.5162
Epoch 5/10
250/250 [==============================] - 116s 465ms/step - loss: 0.6094 - accuracy: 0.6552 - val_loss: 0.6520 - val_accuracy: 0.5864
Epoch 6/10
250/250 [==============================] - 115s 459ms/step - loss: 0.6006 - accuracy: 0.6590 - val_loss: 0.6573 - val_accuracy: 0.6096
Epoch 7/10
250/250 [==============================] - 118s 474ms/step - loss: 0.6017 - accuracy: 0.6562 - val_loss: 0.6510 -